In [613]:
main_information = pd.read_csv("D:/DataSets/msc_datasets_finished/main_information.csv")

In [614]:
addresses = pd.read_csv("D:/DataSets/msc_datasets/addresses.csv")

In [615]:
len(addresses)

2749

In [616]:
addresses_filtered = addresses[addresses['propertyID'].isin(main_information['propertyID'])]
addresses_filtered = addresses_filtered.reset_index(drop=True)

In [617]:
len(addresses_filtered)

2512

In [618]:
main_original = pd.read_csv("D:/DataSets/msc_datasets/main_information_msc.csv")

In [619]:
addresses_new = pd.read_csv("D:/DataSets/msc_datasets_finished/addresses_new.csv")

In [620]:
len(addresses_new)

721

In [621]:
len(main_original)

2512

In [622]:
# check that columns match
if main_original['propertyID'].equals(addresses_filtered['propertyID']):
    print("Columns are identical.")
else:
    print("Columns are not identical.")

Columns are identical.


In [623]:
for i in range(len(main_original)):
    if main_original["propertyID"][i] != addresses_filtered["propertyID"][i]:
        val1 = main_original["propertyID"][i]
        val2 = addresses_filtered["propertyID"][i]
        print(f"matching issue at row {i}, propertyID from main_orginal : {val1}, does not match propertyID from addresses_filtered: {val2}.")

In [624]:
pattern = r'(#\S+|ROOM\s+\S+|APT\s+\S+|UNIT\s+\S+|PENTHOUSE\s+\S+|PH\s+\S+|SUITE\s+\S+|BASEMENT\s+\S+|FLOOR\s+\S+|SPACE\s+\S+|STE\s+\S+)'

# remove the extracted information from the original column
addresses_filtered['streetAddress'] = addresses_filtered['streetAddress'].str.replace(pattern, '', regex = True).str.strip()

In [625]:
addresses_filtered = addresses_filtered.reset_index(drop = True)

In [626]:
unique_addresses = addresses_filtered['streetAddress'].unique()
address_id_mapping = {addresses_filtered: idx for idx, addresses_filtered in enumerate(unique_addresses)}

# Use the map function to assign 'addressID' based on the mapping
addresses_filtered['addressID'] = addresses_filtered['streetAddress'].map(address_id_mapping)

In [627]:
addresses_filtered["agentName"] = main_original["agentName"]
addresses_filtered["postingGroupName"] = main_original["postingGroupName"]

In [628]:
# Create a mapping dictionary from addresses_new
address_mapping = addresses_new.set_index('streetAddress')['neighbourhoodGeoLocation'].to_dict()

# Map the values to addresses_filtered
addresses_filtered['neighbourhoodGeoLocation'] = addresses_filtered['streetAddress'].map(address_mapping)


In [629]:
addresses_filtered = addresses_filtered.drop("zillowBuildingUrl", axis = 1)

In [630]:
addresses_filtered.insert(0,"addressID", addresses_filtered.pop("addressID"))
addresses_filtered.insert(3,"agentName", addresses_filtered.pop("agentName"))

In [631]:
main_information["addressID"] = addresses_filtered["addressID"]

In [632]:
# remove duplicates but keep first
addresses_filtered = addresses_filtered.drop_duplicates(subset='streetAddress', keep='first')
addresses_filtered = addresses_filtered.reset_index(drop = True)

In [633]:
addresses_filtered.loc[:, "geoLocation"] = addresses_filtered["geoLocation"].astype(str)

In [634]:
for i in range(len(addresses_filtered)):
    print(type(addresses_filtered["cityRegion"][i]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [635]:
addresses_filtered['geoLocation'] = addresses_filtered['geoLocation'].str.replace(',', '', regex=True)
addresses_filtered['neighbourhoodGeoLocation'] = addresses_filtered['neighbourhoodGeoLocation'].str.replace(',', '', regex=True)

In [636]:
# Split the geoLocation column into latitude and longitude columns
addresses_filtered[['latitude', 'longitude']] = addresses_filtered['geoLocation'].str.split(' ', expand=True)

# Format the data into the expected SQL format
addresses_filtered['geometry'] = addresses_filtered.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
addresses_filtered = addresses_filtered.drop(columns=['geoLocation', 'latitude', 'longitude'])

In [637]:
# Split the geoLocation column into latitude and longitude columns
addresses_filtered[['latitude', 'longitude']] = addresses_filtered['neighbourhoodGeoLocation'].str.split(' ', expand=True)

# Format the data into the expected SQL format
addresses_filtered['neighbourhoodGeometry'] = addresses_filtered.apply(lambda row: f"POINT({row['latitude']} {row['longitude']})", axis=1)

# Drop the original geoLocation column
addresses_filtered = addresses_filtered.drop(columns=['neighbourhoodGeoLocation', 'latitude', 'longitude'])

In [638]:
addresses_filtered = addresses_filtered.drop("propertyID", axis = 1)

In [639]:
addresses_filtered.insert(1,"geometry", addresses_filtered.pop("geometry"))
addresses_filtered.insert(2,"postingGroupName", addresses_filtered.pop("postingGroupName"))

In [640]:
addresses_filtered.rename(columns={'geometry': 'geoLocation'}, inplace=True)
addresses_filtered.rename(columns={'neighbourhoodGeometry': 'neighbourhoodGeoLocation'}, inplace=True)

In [641]:
addresses_filtered = addresses_filtered[addresses_filtered['addressID'] != 327]

In [642]:
main_information = main_information[main_information['addressID'] != 327]

In [643]:
pd.set_option("display.max_rows", None)
main_information

,propertyID,unitID,propertyDetailsID,priceHistoryID,feeID,addressID,schoolID,buildingID,overview,price,bathrooms,bathroomsHalf,bathroomsFull,bedrooms,livingArea,homeType,homeStatus,datePosted,rentEstimate,availableFrom,availableNow
0,2075342017,0,0,0,0,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,1326,1.0,0,1.0,1.0,850 sqft,Apartment,FOR_RENT,2023-09-20,1346.0,2023-10-27 08:00:00,0
1,2075330457,1,1,1,1,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,1376,1.0,0,1.0,1.0,850 sqft,Apartment,FOR_RENT,2023-09-25,1326.0,2023-11-11 08:00:00,0
2,2075652105,2,2,2,2,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,1629,2.0,0,2.0,2.0,"1,100 sqft",Apartment,FOR_RENT,2023-09-21,1669.0,2023-10-13 08:00:00,0
3,2075119575,3,3,3,3,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,2050,2.0,0,2.0,3.0,"1,315 sqft",Apartment,FOR_RENT,2023-09-28,1758.0,NaN,1
4,2067797682,4,4,4,4,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,1740,2.0,0,2.0,2.0,"1,230 sqft",Apartment,FOR_RENT,2023-09-18,1849.0,2023-11-27 08:00:00,0
5,2071152602,5,5,5,5,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,1850,2.0,0,2.0,2.0,"1,230 sqft",Apartment,FOR_RENT,2023-09-06,1849.0,NaN,1
6,2075652104,6,6,6,6,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,1850,2.0,0,2.0,2.0,"1,230 sqft",Apartment,FOR_RENT,2023-09-11,1849.0,NaN,1
7,2072105027,7,7,7,7,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,1940,2.0,0,2.0,2.0,"1,230 sqft",Apartment,FOR_RENT,2023-09-06,1838.0,NaN,1
8,2062003721,8,8,8,8,0,0,2.505950e+09,Pickwick East's unique family atmosphere is an...,2050,2.0,0,2.0,3.0,"1,315 sqft",Apartment,FOR_RENT,2023-09-12,2120.0,2023-10-20 08:00:00,0
9,2078692687,9,9,9,9,1,1,1.001417e+09,This 680 square foot apartment home has 1 bedr...,1349,1.0,0,1.0,1.0,680 sqft,Apartment,FOR_RENT,2023-09-22,1368.0,NaN,1


In [644]:
addresses_filtered.to_csv("D:/DataSets/backup_finished/addresses.csv", index = False)

In [645]:
main_information.to_csv("D:/DataSets/backup_finished/main_information.csv", index = False)